In [1]:
using Plots, Polynomials

In [2]:
# Define constants
const l = -1       
const zeta = 0.5  

const r0 = 1.0   
const rmin = 0.5
const rmax = 1.5

const r_left = 1.0e-6
const r_right = 5.0

const M = 0.8     
const rho0 = 3 * M/(4 * π * r0^3)  


const nx = 10096
const ng = 2

const threshold = 30.0
const tmin = 0.0
const tmax = 3.0
const C = 0.5;

In [ ]:
current_dir = "/Users/qudx/Documents/ourwork/LQGBHs/Shock Wave/code/X_source/code in paper/Charateristics_FiG_Horizon/"

"/Users/qudx/Documents/ourwork/LQGBHs/Shock Wave/code/X_source/Charateristics_FiG_Horizon/"

# Weak solutions

In [4]:
include(joinpath(current_dir, "initialize_grid.jl"))
include(joinpath(current_dir, "get_initial_bdry.jl"))
include(joinpath(current_dir, "initialize_grid.jl"))
include(joinpath(current_dir, "get_initial_bdry.jl")) 
include(joinpath(current_dir, "get_flux_velocity.jl"))
include(joinpath(current_dir, "get_interface_states.jl"))
include(joinpath(current_dir, "get_PDE_solver.jl"))
include(joinpath(current_dir, "flux_update.jl"))
include(joinpath(current_dir, "modify_sign.jl"))
include(joinpath(current_dir, "update_state.jl"))

gridvalue = InitializeGrid(nx, ng, rmin, rmax);
update_state(gridvalue, C, tmin, tmax, compute_flux, compute_source, compute_velocity, Meff_func, threshold);

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


t = 4.8874013550487596e-5
Flipped sign at position 3
Flipped sign at position 4
Flipped sign at position 5
Flipped sign at position 6
Flipped sign at position 7
Flipped sign at position 8
Flipped sign at position 9
Flipped sign at position 10
Flipped sign at position 11
Flipped sign at position 12
Flipped sign at position 13
Flipped sign at position 14
Flipped sign at position 15
Flipped sign at position 16
Flipped sign at position 17
Flipped sign at position 18
Flipped sign at position 19
Flipped sign at position 20
Flipped sign at position 21
Flipped sign at position 22
Flipped sign at position 23
Flipped sign at position 24
Flipped sign at position 25
Flipped sign at position 26
Flipped sign at position 27
Flipped sign at position 28
Flipped sign at position 29
Flipped sign at position 30
Flipped sign at position 31
Flipped sign at position 32
Flipped sign at position 33
Flipped sign at position 34
Flipped sign at position 35
Flipped sign at position 36
Flipped sign at position 37
F

# Manage Data to have graphs

In [ ]:
using Plots, Base.Threads, Roots

include(joinpath(current_dir, "characteristic_shock.jl"))
include(joinpath(current_dir, "characteristic_original.jl"))

# Compute characteristic values
c1, c2 = c_value(r_left, Meff_func), c_value_outer(r_right, Meff_func)
X_vals = X_inner.(gridvalue.t, 0.7, c1)
t1, t2 = gridvalue.t[findmax(X_vals)[2]], gridvalue.t[findmin(X_vals)[2]]
tb = t1 + (t2 - t1) / 2
Xright = X_outer(gridvalue.x[end], c2)

# Define x indices for characteristic curves
x_indices = collect(3:600:nx)
xvalues = @view gridvalue.x[x_indices]

# Solve characteristic curves in parallel
characteristic_shock = map(x_idx -> solve_r_ode(gridvalue, x_idx), x_indices)

# Define a color palette and ensure consistent coloring
color_palette = [cgrad(:pastel, length(x_indices))[i] for i in 1:length(x_indices)]

# Initialize plot
plotp = plot(xlabel="\$t\$", ylabel="\$r\$", guidefont=14, tickfont=12, palette=:pastel, ylims=(0.35, 1.35))

# Plot shock characteristic curves with consistent colors
for (i, (shock_curve, _)) in enumerate(characteristic_shock)
    plot!(plotp, gridvalue.t, shock_curve, label=false, linewidth=2.0, color=color_palette[i])
end

# Compute and plot shock surface
x_t_poly, _, _, _ = shock_surface(gridvalue, 1, length(gridvalue.t))
t_shock_start_index = findfirst(!iszero, gridvalue.shock_sig)
t_values = @view gridvalue.t[t_shock_start_index:end]
xtshock = [x_t_poly(t) for t in t_values]
plot!(plotp, t_values, xtshock, color=:red, linewidth=1.5, label=false)

# Add vertical and horizontal reference lines
vline!([tb], color=:red, linestyle=:dash, label=false)
annotate!(tb, 0.3, text("\$t_b\$", :red, 12, :center))

ts = gridvalue.t[t_shock_start_index]
vline!([ts], color=:purple, linestyle=:dash, label=false)
annotate!(ts + 0.05, 0.3, text("\$t_s\$", :purple, 12, :center))

# Mark maximum radius
rsmax = maximum(xtshock)
hline!([rsmax], color=:purple, linestyle=:dash, label=false)
annotate!(1.0, rsmax + 0.025, text("\$r_s^{max}\$", :purple, 12, :center))

# Mark `r_b`
xb = find_zero((r -> c2 - 2 * r^3), 0.73, atol=1e-10, rtol=1e-10)
hline!([xb], color=:red, linestyle=:dash, label=false)
annotate!(2.0, 0.7, text("\$r_b\$", :red, 12, :center))

# Compute characteristic curves for the original solver
t_range = collect(0.0:0.01:0.3)
characteristic_curves = map(x_idx -> Characteristic_Original_solver(gridvalue, x_idx, t_range), x_indices)

# Plot original characteristic curves with matching colors
for (i, curve) in enumerate(characteristic_curves)
    plot!(plotp, -curve[1], curve[2], label=false, linewidth=1.5, color=color_palette[i])
end

# Display the plot
display(plotp)
# savefig(joinpath(current_dir, "figures/characteristic_original.png"))

In [ ]:
include(joinpath(current_dir, "get_horizon.jl"))

t_horizon_index = collect(1:100:length(gridvalue.t))
t_value_horizon = [gridvalue.t[i] for i in 1:100:length(gridvalue.t)]
horizons = [find_all_roots(gridvalue, i, -1) for i in t_horizon_index]
h_plus = [horizons[i][1] for i in 1:length(horizons)]
h_minus = [horizons[i][2] for i in 1:length(horizons)];
plot!(t_value_horizon, h_minus, color=:green, label=false, linewidth=1.5)

plot!(t_value_horizon, h_plus, color=:blue, label=false, linewidth=1.5)

annotate!(1, h_plus[1], text("\$h_{+}\$", :blue, 12, :center))
annotate!(1, h_minus[1], text("\$h_{-}\$", :green, 12, :center))
annotate!(1, 1.2, text("Trapped Region", :black, 12, :center))

In [7]:
test_t = collect(0:0.0001:1)
x_i_index = collect(2974:length(gridvalue.x))
curves1 = [Characteristic_Original_solver(gridvalue, i, test_t) for i in x_i_index];

horizon_negative = [find_all_roots_negative(curves1[i][3], curves1[i][2], test_t) for i in 1:length(curves1)]

# Initialize storage vectors
h_plus_negative = []
t_plus_negative = []
h_minus_negative = []
t_minus_negative = []

for i in 1:length(horizon_negative)
    roots = horizon_negative[i]  # Extract the root list for this index

    if length(roots[1]) == 1 
        push!(h_minus_negative, roots[1][1])
        push!(t_minus_negative, -roots[2][1])
    elseif length(roots[1]) == 2
        push!(h_plus_negative, roots[1][1])
        push!(t_plus_negative, -roots[2][1])
        push!(h_minus_negative, roots[1][2])
        push!(t_minus_negative, -roots[2][2])
    end
end

In [ ]:
plot!(t_plus_negative, h_plus_negative, color=:blue, label=false, linewidth=1.5)
plot!(t_minus_negative, h_minus_negative, color=:green, label=false, linewidth=1.5)
# savefig(joinpath(current_dir, "figures/characteristic_shock.png"))